# Byte Pair Encoding (BPE) Example

In [1]:
import datasets
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, processors, trainers

## Load Vietnamese dataset

In [2]:
vi_dataset = datasets.load_dataset("facebook/belebele", "vie_Latn", split="test")
vi_dataset

Dataset({
    features: ['link', 'question_number', 'flores_passage', 'question', 'mc_answer1', 'mc_answer2', 'mc_answer3', 'mc_answer4', 'correct_answer_num', 'dialect', 'ds'],
    num_rows: 900
})

In [3]:
vi_dataset[0]

{'link': 'https://en.wikibooks.org/wiki/Accordion/Right_hand',
 'question_number': 1,
 'flores_passage': 'Hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. Chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. Hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. Trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.',
 'question': 'Theo đoạn văn, đâu không được coi là lời khuyên đúng để chơi được đàn phong cầm?',
 'mc_answer1': 'Để tăng âm lượng, dùng thêm nhiều lực khi ấn phím',
 'mc_answer2': 'Hạn chế tối thiểu động tác thừa để giữ sức',
 'mc_answer3': 'Chú ý khi nhấn các nốt nhạc phải giữ cho tay được thư giãn nhất có thể',
 'mc_answer4': 'Tăng tốc độ di chuyển hộp xếp để tăng âm lượng',
 'correct_answer_num': '1',
 'dialect': 'vie_Latn',
 'ds': '2023-06-01'}

In [4]:
sample_text = vi_dataset[0]["flores_passage"]
sample_text

'Hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. Chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. Hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. Trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.'

## Training BPE

### Init tokenizer

In [5]:
from tokenizers.models import BPE

bpe_tokenizer = Tokenizer(BPE(unk_token="[UNK]",
                            continuing_subword_prefix="_",
                            end_of_word_suffix="_"))

### Normalization

In [6]:
normalizer = normalizers.Sequence([normalizers.NFD(), 
                                normalizers.Lowercase(),
                                normalizers.Strip()])
bpe_tokenizer.normalizer = normalizer

### Pretokenizer

In [7]:
pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(), 
                                        pre_tokenizers.Digits(individual_digits=True)])
bpe_tokenizer.pre_tokenizer = pre_tokenizer

### Train

In [8]:
trainer = trainers.BpeTrainer(special_tokens=["<BLANK>", "<S>", "<UNK>"],
                                    show_progress=True, 
                                    continuing_subword_prefix="_")

In [9]:
def batch_iterator(batch_size=1000):
    for i in range(0, len(vi_dataset), batch_size):
        yield vi_dataset[i : i + batch_size]["flores_passage"]
        
bpe_tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(vi_dataset))

In [11]:
bpe_tokenizer.get_vocab_size()

6207

In [12]:
bpe_tokenizer.get_vocab()

{'_strom': 4953,
 'sepul': 4984,
 '_tro': 3083,
 'antoinette': 5708,
 'chót': 5095,
 'vi': 527,
 'transantarctic': 5806,
 '_ute': 4117,
 '_uit': 4114,
 'revolution': 5727,
 'ipod': 5307,
 'vắt': 5893,
 'gừ': 4886,
 'trừng': 3128,
 'krom': 3872,
 'bò': 3327,
 'zoungla': 5629,
 'paraguay': 3672,
 'nắng': 3627,
 'uppsala': 5602,
 '_á': 138,
 'on': 2962,
 'mede': 6049,
 'khỏe': 1557,
 'gánh': 2889,
 'sy': 3997,
 'ngày': 408,
 '_id': 1052,
 'marta': 4927,
 '_arch': 4524,
 'chase': 5647,
 'đôn': 2598,
 'hạng': 942,
 'buôn': 4691,
 'máu': 1687,
 'chinh': 2000,
 'tham': 679,
 'giác': 1255,
 '_ạch': 754,
 'italo': 5491,
 'dạng': 1060,
 'gripen': 5476,
 'big': 3357,
 'lồng': 3885,
 'quyền': 680,
 'versace': 5250,
 '_xbur': 4235,
 'chiê': 529,
 'educ': 5104,
 '_els': 4628,
 'xưa': 2503,
 'thẻ': 1611,
 'fe': 1785,
 '_vi': 3104,
 'chèo': 2098,
 'ind': 2108,
 'society': 5688,
 'chợt': 3622,
 'tutan': 2611,
 'wned': 4068,
 '_world': 4175,
 '_dina': 4258,
 'ser': 2478,
 

In [13]:
bpe_tokenizer.save('vi-wiki-tokenizer/bpe-tokenizer.json')